### 다익스트라 알고리즘
- 한 지점에서 ~~다른 특정 지점~~ 모든 지점까지의 최단 경로를 구하는 Greedy 알고리즘
- 그래프의 시작점과 끝점을 뒤집어서, 역으로 다른 모든 지점에서 한 지점으로 오는 최단 경로를 구할 수도 있다. (참고: 백준1238)

#### easy version
- $O(V^2)$
    - 총 V번에 걸쳐서 최단 거리가 가장 짧은 노드를 선형 탐색하고, 현재 노드와 연결된 노드들을 매번 일일이 확인하기 때문

In [6]:
n, m = map(int, input().split())

start = int(input())
graph = [[] for i in range(n+1)]
for _ in range(m):
    from_node, to_node, dist = map(int, input().split())
    graph[from_node].append((to_node, dist))
    
visited = [False] * (n+1)
min_dists = [float('inf')] * (n+1)



def get_nearest_node():
    min_value = float('inf')
    index = 0
    for next_node in range(1, n+1):
        if min_dists[next_node] < min_value and not visited[next_node]:
            min_value = min_dists[next_node]
            nearest_node = next_node
    return nearest_node
        
        

def dijkstra(start):
    ## 시작 노드 초기화
    min_dists[start] = 0
    visited[start] = True
    ## 시작노드로 부터 연결된 모든 노드들에 대하여 거리 업데이트
    for to_node, dist in graph[start]:  
        min_dists[to_node] = dist
    
        
    for _ in range(n-1):    ## 남은 n-1개의 노드들에 대하여 반복
        nearest_node = get_nearest_node()   ## 다음으로 확인할 노드는 방문하지 않은 노드 중 가장 가까운 노드
        visited[nearest_node] = True
        for to_node, dist in graph[nearest_node]:
            detouring_dist = min_dists[nearest_node] + dist
            if detouring_dist < min_dists[to_node]:
                min_dists[to_node] = detouring_dist
                
dijkstra(start)

for i in range(1, n+1):
    if min_dists[i] == float('inf'):
        print('infinity')
        
    else:
        print(min_dists[i])

0
2
3
1
2
4


#### efficient one using heap

- $O(E \log V)$
    - E는 간선의 개수, V는 노드의 개수.
    - 현재 노드로 부터 가장 가까운 노드를 힙의 성질로 더 빠르게 찾아서, 시간복잡도를 줄인다!

In [5]:
import heapq

n, m = map(int, input().split())

start = int(input())
graph = [[] for i in range(n+1)]
for _ in range(m):
    from_node, to_node, cost = map(int, input().split())
    graph[from_node].append((to_node, cost))
    
# visited = [False] * (n+1)     ## 힙을 이용한 경우에는 힙이 visited list를 대신한다.
dists = [float('inf')] * (n+1)


def dijkstra(start):
    q = []
    heapq.heappush(q, (0, start))
    dists[start] = 0
    
    while q:
        current_dist, nearest_node = heapq.heappop(q)    ## 가장 거리가 짧은 노드 꺼내기
        
        if dists[nearest_node] < current_dist:   ## 이미 한번 처리된 노드라면 무시하기.  
            continue                             ## 맨 처음 노드에 대해서는 0 < 0이므로, if문 안으로 안들어간다. 즉, <=으로 하면 코드 안돌아감!
        
        for to_node, dist in graph[nearest_node]:
            detouring_dist = current_dist + dist
            if detouring_dist < dists[to_node]:
                dists[to_node] = detouring_dist
                heapq.heappush(q, (detouring_dist, to_node))
                
dijkstra(start)

for i in range(1, n+1):
    if dists[i] == float('inf'):
        print('infinity')
    
    else:
        print(dists[i])
        

0
infinity
infinity
infinity
infinity
infinity


## 플로이드 워셜 알고리즘
- 모든 지점에서 다른 모든 지점까지의 최단 경로를 구하는 DP 알고리즘
- O(N^3) -> 어지간하면 안쓰는게...
    - N개의 노드들에 대하여, 매번 $O(N^2)$의 시간이 소요된다.

In [18]:
n = int(input())
m = int(input())

graph = [[float('inf')] * (n+1) for _ in range(n+1)]
## 대각 성분(자기 자신으로 가는 경우)에 대해서 그래프값 0으로 설정
for from_node in range(1, n+1):
        graph[from_node][from_node] = 0 
## 그래프값 입력받기
for _ in range(m):
    from_node, to_node, dist = map(int, input().split())
    graph[from_node][to_node] = dist


## 알고리즘 수행
for bypass_node in range(1, n+1):
    for from_node in range(1, n+1):
        for to_node in range(1, n+1):
            graph[from_node][to_node] = min(graph[from_node][to_node], graph[from_node][bypass_node] + graph[bypass_node][to_node])


## 결과값 출력
for from_node in range(1, n+1):
    for to_node in range(1, n+1):
        if graph[from_node][to_node] == float('inf'):
            print('inf', end=' ')
        else: print(graph[from_node][to_node], end=' ')
    print()

0
4
8
6
3
0
7
9
5
9
0
4
7
11
2
0


#### 플로이드 워셜 알고리즘에서 k-i-j 순으로 반복을 돌려야하는 이유?
안그러면 누락된다는데, 예시로 정확히 보지는 못했.

## 플로이드 vs 다익스트라

#### 다익스트라 알고리즘은 음의 가중치가 있는 경우, 제대로 동작하지 않는다.
- 이를 보완한 것이 벨만포드 알고리즘이고,
- 플로이드 워셜 알고리즘은 음의 가중치가 있을 때에도 잘 동작하지만,
    - ~~**음수 사이클**, 즉 다른 노드를 거쳐 자신으로 되돌아왔을 때 되려 비용이 감소하는 경우가 존재한다면 유효한 최단경로를 제공할 수 없다.~~ (사실 음수 사이클이 있다면, 최단경로라는 의미가 없다!)

<br>


####  다익스트라는 $O(E \log V)$이고, 플로이드는 $O(V^3)$이다. 그럼 다익스트라를 v번 반복하는 것이 플로이드-와샬보다 더 빠르지 않은가?
- E의 상한이 V^2이기 때문에 다익을 V번 한다면 V^3logV가 될 수 있습니다. E의 제한이 작은 문제의 경우 다익스트라를 V번 돌려서 풀기도 합니다. ([출처](https://www.acmicpc.net/board/view/91454))
- 참고 문제: [백준250250 최고의 간선](https://www.acmicpc.net/problem/25050)


In [7]:
import heapq

heap = []
heapq.heappush(heap, 4)
heapq.heappush(heap, 5)
heapq.heappush(heap, 1)
heapq.heappush(heap, 2)
print(heap)

heapq.heapify(heap)
print(heap)

[1, 2, 4, 5]
[1, 2, 4, 5]
